# Importando os pacotes

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "jupyterlab"
import folium 

# Realizando limpeza dos dados

In [2]:
# Carregando o dataset
df = pd.read_csv('train.csv')
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


In [3]:
df1 = df.copy()

# Retirada de espaços em branco
colunas_texto = ['ID', 'Road_traffic_density', 'Type_of_order', 'Type_of_vehicle', 'City', 'multiple_deliveries', 'Delivery_person_Age']

for coluna in colunas_texto:
    df1[coluna] = df1[coluna].str.strip()


In [4]:
# Removendo linhas com valores inválidos ('NaN' como string) nas colunas
# 'Delivery_person_Age', 'Road_traffic_density' e 'City'
df1 = df1[df1['Road_traffic_density'] != 'NaN']
df1 = df1[df1['City'] != 'NaN']
df1 = df1[df1['Delivery_person_Age'] != 'NaN']
df1 = df1[df1['multiple_deliveries'] != 'NaN']

In [5]:
# Convertendo multiple_deliveries e Delivery_person_Age de texto para numero inteiro (int)
df1['Delivery_person_Age'] = pd.to_numeric(df1['Delivery_person_Age'])
df1['multiple_deliveries'] = pd.to_numeric(df1['multiple_deliveries'])

In [6]:
# Convertendo para tipo data
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format="%d-%m-%Y")

# Convertendo para tipo numérico
df1['Delivery_person_Age'] = pd.to_numeric(df1['Delivery_person_Age'], errors="coerce")
df1['Delivery_person_Ratings'] = pd.to_numeric(df1['Delivery_person_Ratings'], errors="coerce")

In [7]:
# 7. Limpando a coluna de time taken
df1['Time_taken(min)'] = df1['Time_taken(min)'].apply(lambda x: x.split('(min) ')[1])
df1['Time_taken(min)'] = df1['Time_taken(min)'].astype(int)


# A visão Crescimento: Do lado da empresa

## 1. Quantidade de pedidos por dia.

In [8]:
# Preparando o dataset
coluns = ['ID', 'Order_Date']
df_aux = df1.loc[: , coluns].groupby('Order_Date').count().reset_index()
df_aux


,Order_Date,ID
0,2022-02-11,904
1,2022-02-12,794
2,2022-02-13,872
3,2022-02-14,790
4,2022-02-15,860
5,2022-02-16,795
6,2022-02-17,862
7,2022-02-18,785
8,2022-03-01,1031
9,2022-03-02,928


In [9]:
# Fazendo o gráfico

px.bar(df_aux, x='Order_Date', y='ID').show()

## 2. Quantidade de pedidos por semana.

In [10]:
df1['Week_of_year'] = df1['Order_Date'].dt.strftime("%U")
coluns = ['Week_of_year' , 'ID']
df_aux = df1.loc[: , coluns].groupby('Week_of_year').count().reset_index()

px.line(df_aux , x='Week_of_year', y='ID')


## 3. Distribuição dos pedidos por tipo de tráfego.


In [11]:
columns = ['ID', 'Road_traffic_density']
df_aux = df1.loc[:, columns].groupby( 'Road_traffic_density' ).count().reset_index()
df_aux['perc_ID'] = 100 * ( df_aux['ID'] / df_aux['ID'].sum() )

# Gerando o gráfico
px.pie( df_aux, values='perc_ID', names='Road_traffic_density' )

## 4. Comparação do volume de pedidos por cidade e tipo de tráfego.


In [12]:
coluns = ['ID', 'City' , 'Road_traffic_density']
df_aux = df1.loc[: , coluns].groupby(['City' , 'Road_traffic_density']).count().reset_index()
fig = px.scatter(
    df_aux,         # DataFrame com os dados
    x='Road_traffic_density',       # eixo X
    y='City',       # eixo Y
    size='ID', # determina o tamanho das bolhas
)
fig.show()

## 5. A quantidade de pedidos por entregador por semana.

In [13]:
df1['Week_of_year'] = df1['Order_Date'].dt.strftime("%U")
df_aux1 = df1.loc[: , ['Week_of_year' , 'ID']].groupby('Week_of_year').count().reset_index()
df_aux2 = df1.loc[: , ['Week_of_year', 'Delivery_person_ID']].groupby('Week_of_year').nunique().reset_index()
df_aux =  pd.merge(df_aux1 , df_aux2, how='inner')
df_aux['order_by_delivery'] = df_aux['ID'] / df_aux['Delivery_person_ID']

# Gerando o gráfico
fig = px.line(
    df_aux,     # DataFrame com os dados
    x="Week_of_year",   # eixo X
    y="order_by_delivery",   # eixo Y
)
fig.show()

## 6. A localização central de cada cidade por tipo de tráfego.

In [14]:
coluns = ['City','Road_traffic_density','Restaurant_latitude','Restaurant_longitude']
df_aux = df1.loc[: , coluns].groupby(['City','Road_traffic_density']).median().reset_index()
df_aux

,City,Road_traffic_density,Restaurant_latitude,Restaurant_longitude
0,Metropolitian,High,18.539299,76.281128
1,Metropolitian,Jam,18.562450,75.902847
2,Metropolitian,Low,18.551440,75.898497
3,Metropolitian,Medium,18.554382,75.902847
4,Semi-Urban,High,20.423478,76.784976
5,Semi-Urban,Jam,17.434965,76.618496
6,Semi-Urban,Medium,21.160234,77.661931
7,Urban,High,18.580937,75.892574
8,Urban,Jam,18.558416,75.893400
9,Urban,Low,18.546947,75.892840


In [15]:
columns = [
'City',
'Road_traffic_density',
'Delivery_location_latitude',
'Delivery_location_longitude'
]
columns_groupby = ['City', 'Road_traffic_density']
data_plot = df1.loc[:, columns].groupby( columns_groupby ).median().reset_index()
data_plot = data_plot[data_plot['City'] != 'NaN']
data_plot = data_plot[data_plot['Road_traffic_density'] != 'NaN']
# Desenhar o mapa
map_ = folium.Map( zoom_start=11 )
for index, location_info in data_plot.iterrows():
    folium.Marker( [location_info['Delivery_location_latitude'],
location_info['Delivery_location_longitude']],
popup=location_info[['City', 'Road_traffic_density']] ).add_to( map_ )
map_

# 2.0 - Visão: Entregadores 

## 1. A menor e maior idade dos entregadores.

In [9]:
#coluns = ['Delivery_person_Age', 'Delivery_person_ID']
menor_idade = df1['Delivery_person_Age'].min()
maior_idade = df1['Delivery_person_Age'].max()

print(f'A menor idade dos entregadores desta base é de {menor_idade} anos, enquanto a maior é de {maior_idade} anos.')

A menor idade dos entregadores desta base é de 20 anos, enquanto a maior é de 39 anos.


## 2. A pior e a melhor condição de veículos.

In [51]:
df1['Vehicle_condition'].dtype

dtype('int64')

In [52]:
pior_condicao = df1['Vehicle_condition'].min()
melhor_condicao = df1['Vehicle_condition'].max()

print(f'O pior valor de condição dos veículos é {pior_condicao}, enquanto que o melhor valor é {melhor_condicao}.')

O pior valor de condição dos veículos é 0, enquanto que o melhor valor é 2.


## 3. A avaliação média por entregador.

In [54]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30


In [21]:
coluns = ['Delivery_person_ID' , 'Delivery_person_Ratings']
tabela_avaliacoes = df1.loc[: , coluns].groupby('Delivery_person_ID').mean().reset_index().sort_values(by='Delivery_person_Ratings', ascending=False)

tabela_avaliacoes.head()

,Delivery_person_ID,Delivery_person_Ratings
158,AURGRES13DEL03,4.925000
816,KOCRES13DEL01,4.861538
497,GOARES05DEL03,4.842857
959,LUDHRES20DEL03,4.841667
55,AGRRES19DEL02,4.840000


## 4. A avaliação média e o desvio padrão por tipo de tráfego.

In [25]:
coluns = ['Road_traffic_density' , 'Delivery_person_Ratings']
df1.loc[: , coluns].groupby('Road_traffic_density').agg(Delivery_mean=('Delivery_person_Ratings' , 'mean') , Delivery_std=('Delivery_person_Ratings' , 'std')).reset_index() 

,Road_traffic_density,Delivery_mean,Delivery_std
0,High,4.652538,0.272699
1,Jam,4.594428,0.329531
2,Low,4.645947,0.337211
3,Medium,4.660335,0.273956


## 5. A avaliação média e o desvio padrão por condições climáticas.

In [27]:
coluns = ['Weatherconditions' , 'Delivery_person_Ratings']
df1.loc[: , coluns].groupby('Weatherconditions').agg(Delivery_mean=('Delivery_person_Ratings' , 'mean') , Delivery_std=('Delivery_person_Ratings' , 'std')).reset_index() 

,Weatherconditions,Delivery_mean,Delivery_std
0,conditions Cloudy,4.651945,0.281016
1,conditions Fog,4.653087,0.274974
2,conditions Sandstorms,4.612144,0.310527
3,conditions Stormy,4.612569,0.312621
4,conditions Sunny,4.656164,0.395057
5,conditions Windy,4.616844,0.304087


## 6. Os 10 entregadores mais rápidos por cidade.

In [49]:
coluns = ['Delivery_person_ID', 'City' , 'Time_taken(min)']
df_aux = df1.loc[: , coluns].groupby(['City', 'Delivery_person_ID']).mean().sort_values(by=['City', 'Time_taken(min)'] , ascending=True).reset_index()

top_10 = (df_aux.groupby('City')
                 .apply(lambda x: x.nsmallest(10, 'Time_taken(min)'))
                 .reset_index(drop=True))

top_10

/tmp/ipykernel_2695/411788793.py:5: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,KNPRES03DEL02,15.750000
1,Metropolitian,ALHRES02DEL02,17.800000
2,Metropolitian,KNPRES01DEL01,19.125000
3,Metropolitian,KOLRES01DEL03,19.125000
4,Metropolitian,KOCRES02DEL02,19.250000
5,Metropolitian,KOCRES08DEL03,19.800000
6,Metropolitian,KOCRES16DEL03,20.000000
7,Metropolitian,KOCRES02DEL03,20.375000
8,Metropolitian,GOARES14DEL01,20.400000
9,Metropolitian,DEHRES20DEL03,20.428571


## 7. Os 10 entregadores mais lentos por cidade.

In [18]:
colunas = ['Delivery_person_ID' , 'City' , 'Time_taken(min)']

df_aux = df1.loc[: , colunas].groupby(['City' , 'Delivery_person_ID']).mean().sort_values(by=['City', 'Time_taken(min)'] , ascending=False).reset_index()


lentos_10 = (df_aux.groupby('City')
                 .apply(lambda x: x.nlargest(10, 'Time_taken(min)'))
                 .reset_index(drop=True))

lentos_10 

/tmp/ipykernel_1603/2676590400.py:7: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,AGRRES02DEL01,39.428571
1,Metropolitian,AURGRES11DEL03,38.500000
2,Metropolitian,KOLRES03DEL03,38.142857
3,Metropolitian,LUDHRES17DEL03,37.000000
4,Metropolitian,ALHRES18DEL02,36.666667
5,Metropolitian,GOARES08DEL03,36.000000
6,Metropolitian,ALHRES12DEL01,35.000000
7,Metropolitian,AGRRES13DEL02,34.700000
8,Metropolitian,KNPRES06DEL01,34.555556
9,Metropolitian,ALHRES010DEL01,34.500000
